In [4]:
# boilerplate setup
import leafmap.maplibregl as leafmap
import ibis
from ibis import _

conn = ibis.duckdb.connect("tmp")
ca_parquet = "https://data.source.coop/cboettig/ca30x30/ca_areas.parquet"
# or use local copy:
ca_parquet = "/home/rstudio/source.coop/cboettig/ca30x30/ca_areas.parquet"


In [5]:
buffer = -0.00003 

tbl = (
    conn.read_parquet("https://data.source.coop/cboettig/ca30x30/ca_areas.parquet")
    .cast({"SHAPE": "geometry"})
    .rename(geom = "SHAPE")
   # .filter(_.UNIT_NAME == "Angeles National Forest")
    .filter(_.reGAP < 3) 
)
tbl_2023 = tbl.filter(_.Release_Year == 2023).mutate(geom=_.geom.buffer(buffer))
tbl_2024 = tbl.filter(_.Release_Year == 2024)
intersects = tbl_2024.anti_join(tbl_2023, _.geom.intersects(tbl_2023.geom))

new2024 = intersects.select("OBJECTID").mutate(established = 2024)

In [7]:
ca = (conn
      .read_parquet(ca_parquet)
      .cast({"SHAPE": "geometry"})
      .mutate(area = _.SHAPE.area())
      .filter(_.Release_Year == 2024)
      .filter(_.reGAP < 3)
      .left_join(new2024, "OBJECTID")
      .mutate(established=_.established.fill_null(2023))
      .mutate(geom = _.SHAPE.convert("epsg:3310","epsg:4326"))
      .rename(name = "UNIT_NAME", access_type = "ACCESS_TYP", manager = "MNG_AGNCY",
              manager_type = "MNG_AG_LEV", id = "OBJECTID", type = "TYPE")
      .select(_.established, _.reGAP, _.name, _.access_type, _.manager, _.manager_type,
              _.Easement, _.Acres, _.id, _.type, _.geom)
     )
ca.to_parquet("ca2024.parquet")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Using difference (overlap) instead:

In [ ]:
# compute some summary tables:

(ca
 .filter(_.established == 2024)
 .filter(_.manager_type == manager_type)
 .group_by(_.manager, _.manager_type)
 .agg(area = _.Acres.sum())
 .order_by(_.area.desc())
 .execute()
)

In [ ]:
gdf = ca.filter(_.manager == "United States National Park Service", _.established== 2024).execute()
established = {'property': 'established',
               'type': 'categorical',
               'stops': [
                   [2023, "#26542C80"], 
                   [2024, "#F3AB3D80"]]
              }
paint = {"fill-color": established}

m = leafmap.Map(style="positron")
m.add_gdf(gdf,layer_type="fill", name = "CA 30x30", paint = paint)
m.add_layer_control()
m